# Modeling

[Modeling](https://nexus-forge.readthedocs.io/en/latest/interaction.html#modeling) enables the definition and access of data types and properties defined in schemas (e.g JSON templates, [W3C SHACL](https://www.w3.org/TR/shacl)).
Models are configured in the `Model` section of the configuration file.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

## Prefixes
Prefixes are namespaces that are used to put Resource properties within a context.

In [4]:
forge.prefixes()

Used prefixes:
 dc       http://purl.org/dc/elements/1.1/               
 dcat     http://www.w3.org/ns/dcat#                     
 dcterms  http://purl.org/dc/terms/                      
 mba      http://api.brain-map.org/api/v2/data/Structure/
 nsg      https://neuroshapes.org/                       
 owl      http://www.w3.org/2002/07/owl#                 
 prov     http://www.w3.org/ns/prov#                     
 rdf      http://www.w3.org/1999/02/22-rdf-syntax-ns#    
 rdfs     http://www.w3.org/2000/01/rdf-schema#          
 schema   http://schema.org/                             
 sh       http://www.w3.org/ns/shacl#                    
 shsh     http://www.w3.org/ns/shacl-shacl#              
 skos     http://www.w3.org/2004/02/skos/core#           
 vann     http://purl.org/vocab/vann/                    
 void     http://rdfs.org/ns/void#                       
 xsd      http://www.w3.org/2001/XMLSchema#              


## Types
The `type` property of a Resource can be associated to the available types in the Model. These types have a pre-defined set of properties.

In [5]:
forge.types()

Managed entity types:
   - Activity
   - Contribution
   - Dataset
   - Entity
   - Ontology
   - Person


## Templates
The template will provide a set of properties for the givent type that is recomended to be used when creating Resources.

### showing the properties of a type + getting the template of a Mapping for a type

In [6]:
forge.template("Person")

{
    id: ""
}


In [7]:
forge.template("Person", only_required=True)

{
    id: ""
}


### creating (a) Resource instance(s)

#### manually (JSON)

In [8]:
forge.template("Person", output="json", only_required=True)

{
    "id": ""
}


In [9]:
data = {
    "type": "Person",
    "name": "Jane"
}

In [10]:
resource_json = forge.from_json(data)

In [11]:
print(resource_json)

{
    type: Person
    name: Jane
}


#### programmatically (Dict)

In [12]:
template = forge.template("Person", output="dict", only_required=True)

In [13]:
template["name"] = "Jane"

In [14]:
resource_dict = forge.from_json(template)

In [15]:
print(resource_dict)

{
    id: ""
    name: Jane
}


## Validation
It is possible to verify that a Resource is compliant with the suggested type schema available in the Model.

In [16]:
jane = Resource(type="Person", name="Jane Doe")

In [17]:
john = Resource(type="Person", name="John Smith")

In [18]:
persons = [jane, john]

In [19]:
forge.validate(persons)

<count> 2
<action> _validate_many
<succeeded> True


In [20]:
jane._last_action

Action(error=None, message=None, operation='_validate_many', succeeded=True)

In [21]:
jane._validated

True

### automatic status update

In [22]:
jane.email = "jane.doe@epfl.ch"

In [23]:
jane._validated

False

### lazy actions handling

In [24]:
distribution = forge.attach("../../data/persons.csv")

In [25]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution)

In [26]:
forge.validate(jane)

<action> _validate_one
<succeeded> False
<error> ValidationError: resource has lazy actions which need to be executed before


Note: DemoStore doesn't implement file operations yet. Please use another store for the following cell.

In [27]:
forge.validate(jane, execute_actions_before=True)

<action> _validate_one
<succeeded> True


### error handling

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [28]:
mistake = Resource(type="Person")

In [29]:
# resource = Resource(type="Association", agent=mistake)
resource = Resource(type="Dataset", contribution=mistake)

In [30]:
forge.validate(resource)

<action> _validate_one
<succeeded> False
<error> ValidationError: 
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: this11:DatasetShape
	Focus Node: [ <https://neuroshapes.org/contribution> [ rdf:type <http://schema.org/Person> ] ; rdf:type <http://schema.org/Dataset> ]
	Value Node: [ <https://neuroshapes.org/contribution> [ rdf:type <http://schema.org/Person> ] ; rdf:type <http://schema.org/Dataset> ]



In [31]:
resource._last_action

Action(error='ValidationError', message='\nValidation Report\nConforms: False\nResults (1):\nConstraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: this11:DatasetShape\n\tFocus Node: [ <https://neuroshapes.org/contribution> [ rdf:type <http://schema.org/Person> ] ; rdf:type <http://schema.org/Dataset> ]\n\tValue Node: [ <https://neuroshapes.org/contribution> [ rdf:type <http://schema.org/Person> ] ; rdf:type <http://schema.org/Dataset> ]\n', operation='_validate_one', succeeded=False)

In [32]:
resource._validated

False